In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import fasttext
import re

In [2]:
fake = pd.read_csv('fake.csv')
fake.shape

(1339, 2)

In [3]:
non_fake = pd.read_csv('non-fake.csv')
non_fake = non_fake.sample(n=2000,random_state=10)

In [4]:
dataset = pd.concat([fake,non_fake])
dataset.sample(10)

,Review,Label
3863,পুরান ঢাকার গ্যান্ডারিয়াতে new একটা cafe হয়েছে...,1
6369,স্থানঃ মল্লিক কাবাব ঘর (মিরপুর ১১)আইটেমঃ খিরি ...,1
348,কি আর করা!বার বার যেতে হয় তোর মাজারে...... \r\...,0
5623,Ramadan shashTai shash ifter o kora fellam.Pla...,1
610,#DHAMAKA_OFFER \r\n২৫০ টাকার কালারফুল প্লেটার ...,0
4942,ভালোবাসার কাচ্চি খুব খাচ্চি আরাম পাচ্চিসাথে না...,1
455,গভীর রাতে স্বপ্নের উৎপাত! ভয়ংকর স্বপ্ন যাকে বল...,0
72,OFFER-OFFER Realkhadok#Fish_&_Chips #300_𝘁𝗸(re...,0
2637,#জীবন_আছে_যতদিন_খেয়ে_যাব_ততদিন #Best_কা...,1
704,পেইড রিভিউ সোদাও ক্যান ফইন্নি 😑 যে বালের কাচ্চ...,0


In [5]:
dataset.to_csv('bangla_food_review.csv',index=False)

In [6]:
data = pd.read_csv('bangla_food_review.csv')
data.sample(5)

,Review,Label
2717,#২০০_টাকাতে_খিচুড়ি_কালো_ভুনা #...,1
1097,স্থানঃ TFC(The food Company) \r\nআইটেম নাম: \r...,0
38,#Offer_Offer_Offer\r\nOVEN BAKED PASTA \r\nBuy...,0
3323,"RoadSide Kitchen এর পুরো থিম টাই চমকপ্রদ, অবস্...",1
1442,ঠান্ডা ঠান্ডা weather তাও আবার ছুটির দিন!তাই ভ...,1


In [7]:
data.dropna(inplace=True)
data.shape

(3339, 2)

In [8]:
data.Label.replace(1,"Not_Fake",inplace=True)
data.Label.replace(0,"Fake",inplace=True)
data.sample(5)

,Review,Label
1404,ফ্যামিলি অরিয়েন্টেড প্লেস এর সাথে ফ্যামিলি প্ল...,Not_Fake
2510,#বারগার_প্লেটার_বিবরণ #roadside_kitchen বারগার...,Not_Fake
3166,জীবনের সেরা নাগা খেলাম আজ#_নাগা_খাওয়ার_পরে_কি_...,Not_Fake
2649,শহরের কোলাহল থেকে পালিয়ে একদিনের জন্য ঘুরে আস...,Not_Fake
194,আসসালামু আলাইকুম সবাইকে ❤️\r\nঅনেকেই বলেছিলেন ...,Fake


In [9]:
data["Label"] = "__label__" + data["Label"].astype(str)
data.sample(5)

,Review,Label
675,Pizza review 😍\r\nPlace : Backyard kitchen\r\n...,__label__Fake
268,এও কি সম্ভব???\r\nমাত্র ১৩০/= আপনি পাচ্ছেন একট...,__label__Fake
963,#যাহার_পরাণ_মাএ_২০০_টাকায়_খিচুড়ি_নাহি_চায়🙄\r\...,__label__Fake
1270,৭৯৯ টাকায় বিফ সিজলিং থেকে শুরু করে টেবিল ভর্তি...,__label__Fake
2623,একজনের রিভিউ দেখে গেলাম মিরপুর বাসা থেকে একটু ...,__label__Not_Fake


In [10]:
data["category_description"] = data["Label"] +" "+ data["Review"]
data.sample(5)

,Review,Label,category_description
590,#The chillux. \r\nশীতকালীন অফার হিসবে The Chi...,__label__Fake,__label__Fake #The chillux. \r\nশীতকালীন অফার ...
332,🦀 Oshi Sushis - good food you got 🦀\r\nThey a...,__label__Fake,__label__Fake 🦀 Oshi Sushis - good food you g...
1056,Happy New Year 2020\r\nসম্মানিত ভোজনরসিক অতিথি...,__label__Fake,__label__Fake Happy New Year 2020\r\nসম্মানিত ...
3021,পুরান ঢাকায় সকাল সকাল নাস্তা করতে চলে গেলাম বি...,__label__Not_Fake,__label__Not_Fake পুরান ঢাকায় সকাল সকাল নাস্তা...
2616,পূর্বাচলের দিকে যাওয়া হয়না অনেকদিন ! হঠাৎ করেই...,__label__Not_Fake,__label__Not_Fake পূর্বাচলের দিকে যাওয়া হয়না অ...


In [11]:
def preprocess(sentence):
    sentence = re.sub("['।,]+",' ',sentence)
    sentence = re.sub(r'\n', ' ', sentence)
    sentence = re.sub(r'[ ]+', ' ', sentence)
    return sentence.strip().lower()

In [12]:
data.category_description = data.category_description.map(preprocess)
data.sample(5)
data.to_csv("food_review_after_preprocessing.csv",index=False)

In [13]:
data.category_description

0       __label__fake buy one get one mexican পাস্তা \...
1       __label__fake #review দেখে গেছিলাম\r ভাবতেই অব...
2       __label__fake #partner_platter ❤\r #বিগ_বাইট র...
3       __label__fake অথেনটিক পিজ্জা খেতে চাইলে এইটা ম...
4       __label__fake অনলি ফর ইউনিভার্সিটি কাপল স্টুডে...
                              ...                        
3334    __label__not_fake টেবিল ভর্তি খাবার দেখে চলে গ...
3335    __label__not_fake আহারে ভেলপুড়ি এক এর পর এক প...
3336    __label__not_fake আরাবিয়ান ফাস্টফুড ধানমন্ডির ...
3337    __label__not_fake বর্তমান সময়ের সেরা কাচ্চির ক...
3338    __label__not_fake আবার ফিরে যেতে হলো বানকার্স ...
Name: category_description, Length: 3339, dtype: object

In [14]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [15]:
train_set.category_description

3255    __label__not_fake the pabulum #a_great_initiat...
2009    __label__not_fake #৯৯৳তান্দুরী আস্ত একটা মুরগি...
712                            __label__fake #paid review
2861    __label__not_fake কাচ্চি লাভারদের জন্য বর্তমান...
1506    __label__not_fake #১২১টাকা নাকি #২৬টাকা#২৬শে_ম...
                              ...                        
1095    __label__fake place: রাজমহল ফুডকোর্ট \r price:...
1130    __label__fake হ্যালো ভোজন প্রিয় নারায়ণগঞ্জ বা...
1294    __label__fake আচ্ছা! বাম চোখ লাফালে কি বিপদ আস...
860     __label__fake #brownie ❤ \r this fudgy brownie...
3174    __label__not_fake আনন্দময় ঈদের কাছাকাছি সময়ে র...
Name: category_description, Length: 2671, dtype: object

In [16]:
train_set.to_csv('food_review.train',columns=["category_description"],index=False,header=False)
test_set.to_csv('food_review.test',columns=["category_description"],index=False,header=False)

In [17]:
model = fasttext.train_supervised(input='food_review.train')


In [18]:
model.test("food_review.test")

(351, 0.9772079772079773, 0.9772079772079773)

In [19]:
# Step 2: Load the test data
test_data = []
with open('food_review.test', 'r', encoding='utf-8', errors='replace') as f:
    for line in f:
        if ' ' in line:  # Check if there is at least one space in the line
            label, text = line.split(' ', 1)
            test_data.append((label.strip(), text.strip()))
        else:
            print(f"Skipping line due to missing label/text: {line.strip()}")

In [20]:
# Step 3: Get predictions and compare with actual labels
true_negatives = []
true_positives = []
false_negatives = []
false_positives = []

for label, text in test_data:
    prediction = model.predict(text)[0][0]  # Get the predicted label
    
    if label == "__label__not_fake" and prediction == "__label__not_fake":
        true_negatives.append((label, text))
    elif label == "__label__fake" and prediction == "__label__fake":
        true_positives.append((label, text))
    elif label == "__label__fake" and prediction == "__label__not_fake":
        false_negatives.append((label, text))
    elif label == "__label__not_fake" and prediction == "__label__fake":
        false_positives.append((label, text))


In [21]:
model.predict('''○বান্ধুবিরা মিলে অনেক দিন পর বের হয়েছিলাম রিক্সায় ঘুরতে কিন্তুু ওরা খালি চাস্ন থাকে treat জন্য আমার আবার সুন্দর সুন্দর sun glass পরতে অনেক ভাল লাগে নতুন একটা চশমা পরে আজকে বের হয়েছিলাম বান্ধুবিরাও বললো েতাকে মানাইসে treat বাহানা আর খুইজা না পাওয়ায় বলতাসে নতুন glass কিনসস সেই খুশিতে treat দে 😒 কি আর করার অই দিন ফুড ব্যাংকে burger fiesta review দেক্লাম খুব reasonable price তাদের ফুডের আর খাবারও মজা আগেও তাদের ফুড আমি test করেছি 😋then burger fiesta গেলাম আমরা অডার করেছিলাম naga wings 🍗naga wings টা খুবই ঝাল ঝাল আমার খুব মজা লাগে জারা ঝাল খেতে পারেন must be try করবেন and সাথে অবশ্যই cold coffee নিতে হবে তাহলে test টা আরও মজা লাগে 😋 i love this burger fiesta s french fry 🍟🧡তাদের french fry টা খুবই crispy হয় and last one chicken sandwich as a good test 🥪 1.naga wings 4 pics/🍗99 tk. 2.chicken sandwich 4 pics🥪/99 tk. 3.french fry /100 tk 🍟 4.cold coffee 70 tk🍹''')

(('__label__not_fake',), array([0.91422081]))

In [22]:
model.predict('''অথেনটিক পিজ্জা খেতে চাইলে এইটা মাস্ট ট্রাই ❤️ ৪৯ টাকার অফার পিজ্জা খেতে গেছিলাম Pizza Station এ। অফার শেষ, তাই ভাবলাম এসেছি একটা পিজ্জা ট্রাই করেই যাই। অর্ডার করলাম তাদের Beef Supreme নামের পিজ্জাটা। আমার খাওয়া সেরা পিজ্জা গুলোর মধ্যে এইটা একটা। থিন ক্রাস্ট, হালকা চিজ আর উপরে বিফ কিমার টপিং। পুরাই প্রিমিয়াম একটা পিজ্জার ফিল দেয়। বিফ কিমায় হালকা নোনতা ও গোলমরিচ এর স্বাদ আর সাথে দেওয়া মেয়নিসের হালকা মিষ্টি একদম পারফেক্ট ব্যালেন্স। বেশ দারুন লাগে খেতে। যাদের একদম চিজে মাখামাখি পিজ্জা ভালো লাগে না, তারা Pizza Station এর পিজ্জা একবার ট্রাই করে দেখতে পারেন ❤️ দাম: ৪৯০ টাকা (১২") রেটিং: ৯/১০''')

(('__label__not_fake',), array([0.97985798]))

In [19]:
model.save_model('bangla_food_review.model')